# WebTorrent in Google Colab (Perfected Version)

This notebook runs a stable, corrected, and optimized version of the WebTorrent client.

**Instructions:**

1.  **Run the Setup Cell:** Execute the first cell to install all dependencies.
2.  **Get ngrok Authtoken:** Go to [ngrok.com](https://ngrok.com) to get your free authtoken.
3.  **Run the Application Cell:** Execute the second cell and enter your `ngrok` authtoken when prompted.
4.  **Access the UI:** Click the public `ngrok` URL output by the cell to open the WebTorrent Manager.

In [ ]:
#@title 1. Setup Environment
# This cell installs Node.js, webtorrent-cli, and required Python packages.

print('Updating package lists...')
!apt-get update -qq

print('Installing Node.js and npm...')
!apt-get install -y nodejs npm -qq

print('Installing webtorrent-cli...')
!npm install -g webtorrent-cli -qq

print('Installing Python packages...')
!pip install flask flask-socketio pyngrok eventlet -qq

print('
--- Setup Complete ---')

In [ ]:
#@title 2. Run WebTorrent Application (Corrected & Optimized)

import os
import subprocess
import threading
import json
import re
import shutil
import uuid
import time
from pathlib import Path
from flask import Flask, Response, request, send_from_directory
from flask_socketio import SocketIO, emit
from pyngrok import ngrok, conf
from getpass import getpass
import logging

# --- Configuration ---
PORT = 5000
DOWNLOAD_DIR = '/content/downloads'
WEBTORRENT_PATH = shutil.which('webtorrent') or '/usr/bin/webtorrent'

# Disable werkzeug logs to keep output clean
log = logging.getLogger('werkzeug')
log.setLevel(logging.ERROR)

# --- HTML Content ---
HTML_CONTENT = r'''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>WebTorrent Manager</title>
    <link rel="stylesheet" href="/static/styles.css">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&family=JetBrains+Mono:wght@400;500&display=swap" rel="stylesheet">
</head>
<body>
    <div class="app-container">
        <header class="app-header">
            <h1>WebTorrent Manager</h1>
            <div class="header-actions">
                <div class="stats-panel">
                    <div class="stat-item"><i class="fas fa-arrow-down"></i> <span id="total-download-speed">0 KB/s</span></div>
                    <div class="stat-item"><i class="fas fa-arrow-up"></i> <span id="total-upload-speed">0 KB/s</span></div>
                </div>
                <div class="connection-status" id="connection-status"><span class="status-indicator" id="status-indicator"></span> <span id="status-text">Connecting...</span></div>
            </div>
        </header>
        <main class="app-main">
            <section class="add-torrent-section">
                <div class="input-group">
                    <div class="input-wrapper"><i class="fas fa-magnet input-icon"></i> <input type="text" id="magnet-input" placeholder="Paste magnet link or torrent URL here..." /></div>
                    <div class="button-group"><button id="start-download-btn" class="btn btn-primary"><i class="fas fa-download"></i> Download</button></div>
                </div>
            </section>
            <section class="torrents-section">
                <div class="section-header"><h2>Active Torrents</h2></div>
                <div id="torrents-container" class="torrents-container"><div class="no-torrents" id="no-torrents"><i class="fas fa-inbox fa-2x"></i><p>No active torrents. Add one to get started.</p></div></div>
            </section>
        </main>
    </div>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.0.1/socket.io.js"></script>
    <script src="/static/app.js"></script>
</body>
</html>
'''

# --- CSS Content ---
CSS_CONTENT = r'''
* { margin: 0; padding: 0; box-sizing: border-box; }
:root { --primary: #6366f1; --primary-dark: #4f46e5; --primary-light: #818cf8; --secondary: #10b981; --bg-dark: #0f172a; --bg-medium: #1e293b; --bg-light: #334155; --text-primary: #f8fafc; --text-secondary: #cbd5e1; --text-muted: #94a3b8; --border: #334155; --success: #22c55e; --danger: #ef4444; --warning: #f59e0b; --info: #3b82f6; --shadow: 0 4px 6px -1px rgba(0,0,0,0.1), 0 2px 4px -1px rgba(0,0,0,0.06); --space-2: 0.5rem; --space-3: 0.75rem; --space-4: 1rem; --space-6: 1.5rem; }
body { font-family: 'Inter', sans-serif; background: var(--bg-dark); color: var(--text-primary); line-height: 1.6; }
.app-container { max-width: 1400px; margin: 0 auto; padding: var(--space-6); }
.app-header { background: var(--bg-medium); border-radius: 12px; padding: var(--space-6); margin-bottom: var(--space-6); display: flex; justify-content: space-between; align-items: center; box-shadow: var(--shadow); border: 1px solid var(--border); }
.app-header h1 { font-size: 1.75rem; }
.header-actions { display: flex; align-items: center; gap: var(--space-4); }
.stats-panel { display: flex; gap: var(--space-4); }
.stat-item { display: flex; align-items: center; gap: var(--space-2); }
.connection-status { display: flex; align-items: center; gap: var(--space-2); padding: var(--space-2) var(--space-4); border-radius: 20px; background: var(--bg-dark); border: 1px solid var(--border); }
.status-indicator { width: 10px; height: 10px; border-radius: 50%; background: var(--warning); }
.status-indicator.connected { background: var(--success); }
.status-indicator.disconnected { background: var(--danger); }
.app-main { display: flex; flex-direction: column; gap: var(--space-6); }
.add-torrent-section, .torrents-section { background: var(--bg-medium); border-radius: 12px; padding: var(--space-6); box-shadow: var(--shadow); border: 1px solid var(--border); }
.input-group { display: flex; gap: var(--space-4); align-items: center; }
.input-wrapper { flex: 1; position: relative; }
.input-icon { position: absolute; left: var(--space-4); top: 50%; transform: translateY(-50%); color: var(--text-muted); }
#magnet-input { width: 100%; padding: var(--space-3) var(--space-4) var(--space-3) 2.5rem; background: var(--bg-dark); border: 1px solid var(--border); border-radius: 8px; font-size: 1rem; color: var(--text-primary); }
.btn { padding: var(--space-3) var(--space-4); border: none; border-radius: 8px; font-weight: 600; cursor: pointer; display: inline-flex; align-items: center; gap: var(--space-2); }
.btn-primary { background: var(--primary); color: white; }
.section-header { margin-bottom: var(--space-4); }
.torrents-container { display: grid; grid-template-columns: repeat(auto-fill, minmax(400px, 1fr)); gap: var(--space-6); }
.no-torrents { text-align: center; color: var(--text-muted); padding: var(--space-6); grid-column: 1 / -1; }
.torrent-card { background: var(--bg-light); border-radius: 12px; padding: var(--space-4); display: flex; flex-direction: column; gap: var(--space-3); border: 1px solid var(--border); }
.torrent-header { display: flex; justify-content: space-between; align-items: flex-start; }
.torrent-name { font-weight: 600; word-break: break-all; }
.torrent-actions button { background: none; border: 1px solid var(--border); color: var(--text-secondary); padding: var(--space-2); border-radius: 6px; }
.progress-bar { width: 100%; background: var(--bg-dark); height: 8px; border-radius: 4px; overflow: hidden; }
.progress-bar-inner { height: 100%; background: var(--primary); width: 0%; transition: width 0.3s ease; }
.torrent-stats { display: grid; grid-template-columns: 1fr 1fr; gap: var(--space-2); font-size: 0.875rem; color: var(--text-secondary); }
.stream-url-container { display: flex; align-items: center; justify-content: space-between; background-color: var(--bg-dark); padding: var(--space-3); margin-top: var(--space-2); border-radius: 8px; }
.stream-url-link { color: var(--primary-light); text-decoration: none; font-family: 'JetBrains Mono', monospace; white-space: nowrap; overflow: hidden; text-overflow: ellipsis; }
.btn-copy-url { background: var(--primary); color: white; border-radius: 6px; padding: 6px 10px; }
.file-list { list-style: none; margin-top: var(--space-3); padding: 0; max-height: 150px; overflow-y: auto; }
.file-item { display: flex; justify-content: space-between; align-items: center; padding: var(--space-2) 0; border-bottom: 1px solid var(--border); }
.file-name { flex: 1; white-space: nowrap; overflow: hidden; text-overflow: ellipsis; }
.file-actions { display: flex; gap: var(--space-2); }
'''

# --- JavaScript Content ---
JS_CONTENT = r'''
document.addEventListener('DOMContentLoaded', () => {
    const socket = io();
    const magnetInput = document.getElementById('magnet-input');
    const startDownloadBtn = document.getElementById('start-download-btn');
    const torrentsContainer = document.getElementById('torrents-container');
    const noTorrents = document.getElementById('no-torrents');
    const statusText = document.getElementById('status-text');
    const statusIndicator = document.getElementById('status-indicator');

    socket.on('connect', () => {
        statusText.textContent = 'Connected';
        statusIndicator.className = 'status-indicator connected';
        socket.emit('get-torrents');
    });

    socket.on('disconnect', () => {
        statusText.textContent = 'Disconnected';
        statusIndicator.className = 'status-indicator disconnected';
    });

    startDownloadBtn.addEventListener('click', () => {
        const magnet = magnetInput.value.trim();
        if (magnet) {
            socket.emit('add-torrent', { magnet });
            magnetInput.value = '';
        }
    });

    socket.on('all-torrents', (torrents) => {
        torrentsContainer.innerHTML = '';
        if (Object.keys(torrents).length === 0) {
            torrentsContainer.appendChild(noTorrents);
        } else {
            for (const torrentId in torrents) {
                updateTorrentCard(torrents[torrentId]);
            }
        }
    });

    socket.on('torrent-update', (data) => {
        updateTorrentCard(data);
    });
    
    socket.on('torrent-removed', (data) => {
        const card = document.getElementById(`torrent-${data.torrentId}`);
        if (card) {
            card.remove();
        }
        if (torrentsContainer.children.length === 1 && torrentsContainer.contains(noTorrents)) {
            noTorrents.style.display = 'block';
        }
    });

    function formatBytes(bytes, decimals = 2) {
        if (!bytes || bytes === 0) return '0 Bytes';
        const k = 1024;
        const dm = decimals < 0 ? 0 : decimals;
        const sizes = ['Bytes', 'KB', 'MB', 'GB', 'TB'];
        const i = Math.floor(Math.log(bytes) / Math.log(k));
        return parseFloat((bytes / Math.pow(k, i)).toFixed(dm)) + ' ' + sizes[i];
    }

    function updateTorrentCard(torrent) {
        let card = document.getElementById(`torrent-${torrent.torrentId}`);
        if (!card) {
            if(noTorrents) noTorrents.style.display = 'none';
            card = document.createElement('div');
            card.className = 'torrent-card';
            card.id = `torrent-${torrent.torrentId}`;
            torrentsContainer.appendChild(card);
        }

        const filesHTML = (torrent.files || []).map((file, index) => {
            let streamUrl = '#';
            if (torrent.streamingUrl) {
                // Correctly construct per-file stream URL
                try {
                    const streamUrlObj = new URL(torrent.streamingUrl);
                    streamUrlObj.pathname = `/${index}/${encodeURIComponent(file.name)}`;
                    streamUrl = streamUrlObj.href;
                } catch (e) { console.error('Invalid streaming URL', torrent.streamingUrl); }
            }
            return `
            <li class="file-item">
                <span class="file-name" title="${file.path}">${file.name} (${formatBytes(file.length)})</span>
                <div class="file-actions">
                    <a href="${streamUrl}" target="_blank" class="btn btn-primary btn-sm">Stream</a>
                </div>
            </li>`
        }).join('');

        card.innerHTML = `
            <div class="torrent-header">
                <div class="torrent-name">${torrent.name || torrent.torrentId}</div>
                <div class="torrent-actions"><button class="stop-btn btn btn-danger btn-sm">Stop</button></div>
            </div>
            <div class="progress-bar"><div class="progress-bar-inner" style="width: ${torrent.progress || 0}%"></div></div>
            <div class="torrent-stats">
                <div>Status: ${torrent.status || 'connecting'}</div>
                <div>Peers: ${torrent.peers || 0}</div>
                <div>Down: ${formatBytes(torrent.downloadSpeed || 0)}/s</div>
                <div>Up: ${formatBytes(torrent.uploadSpeed || 0)}/s</div>
            </div>
            ${torrent.streamingUrl ? `
            <div class="stream-url-container">
                <a href="${torrent.streamingUrl}" target="_blank" class="stream-url-link">${torrent.streamingUrl}</a>
                <button class="btn-copy-url btn btn-sm">Copy</button>
            </div>` : ''}
            ${filesHTML ? `<ul class="file-list">${filesHTML}</ul>` : ''}
        `;

        card.querySelector('.stop-btn').addEventListener('click', () => socket.emit('stop-torrent', { torrentId: torrent.torrentId }));
        const copyBtn = card.querySelector('.btn-copy-url');
        if(copyBtn) {
            copyBtn.addEventListener('click', (e) => {
                navigator.clipboard.writeText(torrent.streamingUrl);
                e.target.textContent = 'Copied!';
                setTimeout(() => { e.target.textContent = 'Copy'; }, 2000);
            });
        }
    }
});
'''

# --- Flask App and Torrent Manager ---
app = Flask(__name__, static_folder=None)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app, async_mode='eventlet')

class TorrentManager:
    def __init__(self):
        self.active_torrents = {}
        self.lock = threading.Lock()
        os.makedirs(DOWNLOAD_DIR, exist_ok=True)

    def get_sanitized_torrents(self):
        sanitized = {}
        with self.lock:
            for tid, data in self.active_torrents.items():
                # Create a copy and remove the non-serializable 'process' object
                clean_data = data.copy()
                del clean_data['process']
                sanitized[tid] = clean_data
        return sanitized

    def start_torrent(self, magnet_link):
        torrent_id = str(uuid.uuid4())[:8]
        # Use --port -1 to find a random available port
        command = [WEBTORRENT_PATH, 'download', magnet_link, '--out', DOWNLOAD_DIR, '--keep-seeding', '--port', '-1']
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, bufsize=1, encoding='utf-8', errors='ignore')
        
        status = {
            'torrentId': torrent_id, 'process': process, 'status': 'starting', 'name': 'Fetching metadata...',
            'files': [], 'downloadSpeed': 0, 'uploadSpeed': 0, 'progress': 0, 'peers': 0, 'streamingUrl': None
        }
        
        with self.lock:
            self.active_torrents[torrent_id] = status
        
        thread = threading.Thread(target=self._listen_to_process, args=(torrent_id, process))
        thread.daemon = True
        thread.start()
        return torrent_id

    def stop_torrent(self, torrent_id):
        with self.lock:
            if torrent_id in self.active_torrents:
                print(f"Stopping torrent {torrent_id}...")
                self.active_torrents[torrent_id]['process'].terminate()
                self.active_torrents[torrent_id]['process'].wait() # Ensure process is killed
                del self.active_torrents[torrent_id]
                socketio.emit('torrent-removed', {'torrentId': torrent_id})
                print(f"Torrent {torrent_id} stopped and removed.")

    def _listen_to_process(self, torrent_id, process):
        for line in iter(process.stdout.readline, ''):
            self.parse_cli_output(torrent_id, line.strip())
        process.stdout.close()
        # Handle process exit
        with self.lock:
            if torrent_id in self.active_torrents:
                status = self.active_torrents[torrent_id]
                status['status'] = 'finished'
                # Create a clean copy for emitting
                clean_status = status.copy()
                del clean_status['process']
                socketio.emit('torrent-update', clean_status)

    def parse_cli_output(self, torrent_id, line):
        with self.lock:
            if torrent_id not in self.active_torrents: return
            status = self.active_torrents[torrent_id]
            updated = False

            if 'Downloading:' in line and status['name'] == 'Fetching metadata...':
                status['name'] = line.split('Downloading:')[1].strip()
                updated = True
            
            if 'Server running at:' in line and not status['streamingUrl']:
                base_url = line.split('Server running at:')[1].strip()
                try:
                    http_tunnels = [t for t in ngrok.get_tunnels() if t.proto == 'https']
                    if http_tunnels:
                        public_url = http_tunnels[0].public_url
                        status['streamingUrl'] = base_url.replace('http://localhost:8000', public_url) # webtorrent-cli default port is 8000
                    else:
                        status['streamingUrl'] = base_url # Fallback
                except Exception:
                    status['streamingUrl'] = base_url # Fallback
                updated = True

            if 'Speed:' in line:
                status['status'] = 'downloading'
                parts = line.split()
                try:
                    progress = float(parts[parts.index('%') - 1].replace('%', ''))
                    download_speed = self.parse_speed_string(parts[parts.index('Down:')+1] + " " + parts[parts.index('Down:')+2])
                    upload_speed = self.parse_speed_string(parts[parts.index('Up:')+1] + " " + parts[parts.index('Up:')+2])
                    peers = int(parts[parts.index('Peers:')+1])
                    
                    if (status['progress'] != progress or status['downloadSpeed'] != download_speed or 
                        status['uploadSpeed'] != upload_speed or status['peers'] != peers):
                        status['progress'] = progress
                        status['downloadSpeed'] = download_speed
                        status['uploadSpeed'] = upload_speed
                        status['peers'] = peers
                        updated = True
                except (ValueError, IndexError): pass

            if line.startswith('- ') and not any(f['path'] == line[2:].split(' (')[0] for f in status['files']):
                match = re.search(r'- (.*) \((.*)\)', line)
                if match:
                    status['files'].append({'path': match.group(1), 'name': os.path.basename(match.group(1)), 'length': self.parse_size_string(match.group(2))})
                    updated = True
            
            if updated:
                clean_status = status.copy()
                del clean_status['process']
                socketio.emit('torrent-update', clean_status)

    def parse_speed_string(self, s):
        s = s.lower().replace('/s', '')
        if 'kb' in s: return float(s.replace('kb', '').strip()) * 1024
        if 'mb' in s: return float(s.replace('mb', '').strip()) * 1024 * 1024
        if 'gb' in s: return float(s.replace('gb', '').strip()) * 1024 * 1024 * 1024
        return 0

    def parse_size_string(self, s):
        s = s.lower()
        if 'kb' in s: return float(s.replace('kb', '').strip()) * 1024
        if 'mb' in s: return float(s.replace('mb', '').strip()) * 1024 * 1024
        if 'gb' in s: return float(s.replace('gb', '').strip()) * 1024 * 1024 * 1024
        return 0

torrent_manager = TorrentManager()

@app.route('/')
def index():
    return Response(HTML_CONTENT, mimetype='text/html')

@app.route('/static/<path:path>')
def send_static(path):
    if path == 'styles.css':
        return Response(CSS_CONTENT, mimetype='text/css')
    elif path == 'app.js':
        return Response(JS_CONTENT, mimetype='application/javascript')
    return '', 404

@socketio.on('connect')
def handle_connect():
    print("Client connected")
    emit('all-torrents', torrent_manager.get_sanitized_torrents())

@socketio.on('get-torrents')
def handle_get_torrents():
    emit('all-torrents', torrent_manager.get_sanitized_torrents())

@socketio.on('add-torrent')
def handle_add_torrent(data):
    magnet = data.get('magnet')
    if magnet:
        print(f"Adding torrent: {magnet[:30]}...")
        torrent_manager.start_torrent(magnet)

@socketio.on('stop-torrent')
def handle_stop_torrent(data):
    torrent_id = data.get('torrentId')
    if torrent_id:
        torrent_manager.stop_torrent(torrent_id)

# --- Main Execution ---
if __name__ == '__main__':
    try:
        print('Please enter your ngrok authtoken.')
        print('You can get it from https://dashboard.ngrok.com/get-started/your-authtoken')
        authtoken = getpass('Authtoken: ')
        conf.get_default().auth_token = authtoken
        
        # Disconnect any existing tunnels
        for t in ngrok.get_tunnels():
            ngrok.disconnect(t.public_url)

        public_url = ngrok.connect(PORT, 'http')
        print(f'
--- WebTorrent UI is running at: {public_url} ---
')

        socketio.run(app, port=PORT, log_output=False)

    except Exception as e:
        print(f'An error occurred: {e}')
    finally:
        print('Shutting down...')
        for t in ngrok.get_tunnels():
            ngrok.disconnect(t.public_url)
        for torrent_id in list(torrent_manager.active_torrents.keys()):
            torrent_manager.stop_torrent(torrent_id)
        print('Shutdown complete.')